In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy import stats
import torch
import json
import random
from tqdm import tqdm
from collections import defaultdict
import re
import shutil

In [2]:
base_dir_path = '/project/CollabRoboGroup/datasets/official_data'
df =  pd.read_csv(f'{base_dir_path}/test_except_tiny_object_other_con.csv')
df.head()

,id,ego_view_image,exo_view_image,top_view_image,ego_view_image_object_start_point_x,ego_view_image_object_start_point_y,ego_view_image_object_end_point_x,ego_view_image_object_end_point_y,exo_view_image_object_start_point_x,exo_view_image_object_start_point_y,...,is_contrastive,view_name,setting_name,object_category,absolute_location_ego,absolute_location_exo,spatial_relation_ego,spatial_relation_exo,instruction_template,perspective
0,4757_1,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/eg...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/ex...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,0,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,template_1_1,NaN
1,4757_2,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/eg...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/ex...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,0,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,ego_template_2_2,ego
2,4757_3,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/eg...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/ex...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,0,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,exo_template_3_1,exo
3,4757_4,5.10.2022.1.00.06.AM_4757/only_gaze/ego_view/c...,5.10.2022.1.00.06.AM_4757/only_gaze/exo_view/c...,5.10.2022.1.00.06.AM_4757/only_gaze/top_view/c...,379.255157,258.922913,413.606934,275.765198,236.839050,314.197342,...,0,top_view,only_gaze,Pistol,left,right,right of,left of,template_1_2,NaN
4,4757_4_713_53_only_gaze,5.10.2022.1.00.17.PM_713/only_gaze/ego_view/ca...,5.10.2022.1.00.17.PM_713/only_gaze/exo_view/ca...,5.10.2022.1.00.17.PM_713/only_gaze/top_view/ca...,329.474030,225.344101,372.213959,270.294693,573.024536,273.194641,...,1,top_view,wrong_only_gaze,Table Lamp,right,left,left of,right of,template_1_2,NaN


In [3]:
df.shape

(121262, 37)

In [15]:
df['instruction_template'].unique()

array(['template_1_1', 'ego_template_2_2', 'exo_template_3_1',
       'template_1_2', 'ego_template_2_1', 'ego_template_3_2',
       'template_null', 'exo_template_3_2', 'ego_template_3_1',
       'exo_template_2_2', 'exo_template_2_1'], dtype=object)

In [16]:
def is_choose():
    random_choice = random.randint(0, 1)
    if(random_choice==0):
        return True #skip the data
    return False

In [17]:
df['setting_name'].unique()

array(['both_gaze_gesture', 'only_gaze', 'wrong_only_gaze',
       'only_gesture', 'wrong_only_gesture', 'only_objects',
       'wrong_only_objects', 'wrong_gaze_gesture'], dtype=object)

In [18]:
mturk_base_dir_path = '/project/CollabRoboGroup/datasets'
setting_names = ['both_gaze_gesture', 'only_gaze', 'wrong_only_gaze',
       'only_gesture', 'wrong_only_gesture', 'only_objects',
       'wrong_only_objects', 'wrong_gaze_gesture']
view_names = ['ego_view_image', 'exo_view_image', 'top_view_image']

stat = {}
for setting_name in setting_names:
    for template_no in ['1', '2', '3']:
        for i in [0,1]:
            stat[f'{setting_name}_{template_no}_valid_{i}_single'] = 50
            stat[f'{setting_name}_{template_no}_valid_{i}_multi'] = 50

seleted_data = []
view_types = []
for i, row in df.iterrows():
    view_type = 'single'
    if(is_choose()):
        view_type = 'multi'
    
    template_no = row['instruction_template']
    if('null' in template_no):
        template_no = 'null'
    elif(('1_1' in template_no) or ('1_2' in template_no)):
        template_no = '1'
    else:
        template_no = template_no.split('_')[2]
    if(template_no=='null'):
        seleted_data.append(0)
        view_types.append('NA')
        continue
    stat_key = f'{row["setting_name"]}_{template_no}_valid_{row["is_contrastive"]}_{view_type}'
    if(stat[stat_key]>0):
        seleted_data.append(1)
        data_id = row['id']
        view_types.append(view_type)
        if(view_type=='multi'):
            
            for view_name in view_names:
                filepath = row[view_name]
                filename = filepath.split('/')[-1]
#                 print('#########', filepath)
                new_filename = f'{row["id"]}_{view_type}_{setting_name}_{view_name}_{filename}'
                source_path = f'{base_dir_path}/{filepath}'
                des_path = f'{mturk_base_dir_path}/embodied_mturk_study_final/{new_filename}'
                df.at[i, view_name] = f'https://mm-grounding.s3.amazonaws.com/embodied_mturk_study_final/{new_filename}'
                try:
#                     print(source_path, des_path)
                    shutil.copy(source_path, des_path)
                except Exception as e:
#                     print('************', filepath)
#                     print(source_path, des_path)
                    print('file copy is not successfull', e)
                    continue

        else:
            view_name = view_names[random.randint(0, 2)]
            filepath = row[view_name]
            filename = filepath.split('/')[-1]

            new_filename = f'{row["id"]}_{view_type}_{setting_name}_{view_name}_{filename}'
            source_path = f'{base_dir_path}/{filepath}'
            des_path = f'{mturk_base_dir_path}/embodied_mturk_study_final/{new_filename}'
            df.at[i, view_name] = f'https://mm-grounding.s3.amazonaws.com/embodied_mturk_study_final/{new_filename}'
            try:
#                 print(source_path, des_path)
                shutil.copy(source_path, des_path)
            except Exception as e:
                print('file copy is not successfull', e)
                continue

            for vn in view_names:
                if(vn!=view_name):
                    df.at[i, vn] = 'None'

        
        stat[stat_key] -= 1
    else:
        seleted_data.append(0)
        view_types.append('NA')
    if(i%10000==0):
        print(f'copy completed {i}')  

copy completed 0
copy completed 10000
copy completed 20000
copy completed 30000
copy completed 40000
copy completed 60000
copy completed 70000
copy completed 80000
copy completed 90000
copy completed 100000
copy completed 110000
copy completed 120000


In [19]:
print(len(seleted_data), df.shape)

121262 (121262, 37)


In [20]:
df['is_selected_for_mturk'] = seleted_data
df['view_type'] = view_types
df.head()

,id,ego_view_image,exo_view_image,top_view_image,ego_view_image_object_start_point_x,ego_view_image_object_start_point_y,ego_view_image_object_end_point_x,ego_view_image_object_end_point_y,exo_view_image_object_start_point_x,exo_view_image_object_start_point_y,...,setting_name,object_category,absolute_location_ego,absolute_location_exo,spatial_relation_ego,spatial_relation_exo,instruction_template,perspective,is_selected_for_mturk,view_type
0,4757_1,None,None,https://mm-grounding.s3.amazonaws.com/embodied...,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,both_gaze_gesture,Pistol,left,right,right of,left of,template_1_1,NaN,1,single
1,4757_2,https://mm-grounding.s3.amazonaws.com/embodied...,None,None,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,both_gaze_gesture,Pistol,left,right,right of,left of,ego_template_2_2,ego,1,single
2,4757_3,None,None,https://mm-grounding.s3.amazonaws.com/embodied...,375.997650,270.047318,410.124390,287.017059,236.839050,314.197342,...,both_gaze_gesture,Pistol,left,right,right of,left of,exo_template_3_1,exo,1,single
3,4757_4,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,379.255157,258.922913,413.606934,275.765198,236.839050,314.197342,...,only_gaze,Pistol,left,right,right of,left of,template_1_2,NaN,1,multi
4,4757_4_713_53_only_gaze,None,https://mm-grounding.s3.amazonaws.com/embodied...,None,329.474030,225.344101,372.213959,270.294693,573.024536,273.194641,...,wrong_only_gaze,Table Lamp,right,left,left of,right of,template_1_2,NaN,1,single


In [21]:
tm_df= df[df['is_selected_for_mturk']==1]
df_view = {}
print(tm_df.shape)
for view_type in ['single', 'multi']:
    df_view[view_type] = tm_df[tm_df['view_type']==view_type]
    print(df_view[view_type].shape)

(2400, 39)
(1200, 39)
(1200, 39)


In [25]:
df_view['single'].groupby('is_contrastive').count()

,id,ego_view_image,exo_view_image,top_view_image,ego_view_image_object_start_point_x,ego_view_image_object_start_point_y,ego_view_image_object_end_point_x,ego_view_image_object_end_point_y,exo_view_image_object_start_point_x,exo_view_image_object_start_point_y,...,setting_name,object_category,absolute_location_ego,absolute_location_exo,spatial_relation_ego,spatial_relation_exo,instruction_template,perspective,is_selected_for_mturk,view_type
is_contrastive,,,,,,,,,,,,,,,,,,,,,
0,600,600,600,600,600,600,600,600,600,600,...,600,600,524,524,600,600,600,400,600,600
1,600,600,600,600,600,600,600,600,600,600,...,600,600,522,522,600,600,600,100,600,600


In [24]:
mturk_base_dir_path = '/project/CollabRoboGroup/datasets/embodied_mturk_study_final'
for view_type in ['single', 'multi']:
    df_view[view_type].to_csv(f'{mturk_base_dir_path}/data_{view_type}.csv', index=False)

In [4]:
mturk_base_dir_path = '/project/CollabRoboGroup/datasets/embodied_mturk_study_final'
tm = pd.read_csv(f'{mturk_base_dir_path}/data_multi.csv')
tm.head()

,id,ego_view_image,exo_view_image,top_view_image,ego_view_image_object_start_point_x,ego_view_image_object_start_point_y,ego_view_image_object_end_point_x,ego_view_image_object_end_point_y,exo_view_image_object_start_point_x,exo_view_image_object_start_point_y,...,setting_name,object_category,absolute_location_ego,absolute_location_exo,spatial_relation_ego,spatial_relation_exo,instruction_template,perspective,is_selected_for_mturk,view_type
0,4757_4,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,379.255157,258.922913,413.606934,275.765198,236.839050,314.197342,...,only_gaze,Pistol,left,right,right of,left of,template_1_2,NaN,1,multi
1,4757_5,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,379.255157,258.922913,413.606934,275.765198,236.839050,314.197342,...,only_gaze,Pistol,left,right,right of,left of,ego_template_2_1,ego,1,multi
2,4757_5_713_53_only_gaze,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,329.474030,225.344101,372.213959,270.294693,573.024536,273.194641,...,wrong_only_gaze,Table Lamp,right,left,left of,right of,ego_template_2_1,NaN,1,multi
3,4757_8,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,487.103546,369.844223,534.230835,396.264565,236.839050,314.197342,...,only_gesture,Pistol,left,right,right of,left of,template_1_2,NaN,1,multi
4,4757_8_713_57_only_gesture,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,https://mm-grounding.s3.amazonaws.com/embodied...,173.067612,295.367767,232.231339,351.176056,573.024536,273.194641,...,wrong_only_gesture,Table Lamp,right,left,left of,right of,template_1_2,NaN,1,multi


In [7]:
print(tm.loc[10, 'ego_view_image'])
print(tm.loc[10, 'exo_view_image'])
print(tm.loc[10, 'top_view_image'])

https://mm-grounding.s3.amazonaws.com/embodied_mturk_study_final/4757_17_multi_wrong_gaze_gesture_ego_view_image_screen_720x480_2022-05-10_01-00-26.png
https://mm-grounding.s3.amazonaws.com/embodied_mturk_study_final/4757_17_multi_wrong_gaze_gesture_exo_view_image_screen_720x480_2022-05-10_01-00-26.png
https://mm-grounding.s3.amazonaws.com/embodied_mturk_study_final/4757_17_multi_wrong_gaze_gesture_top_view_image_screen_720x480_2022-05-10_01-00-26.png


In [6]:
mturk_base_dir_path = '/project/CollabRoboGroup/datasets'
setting_names = ['both_gaze_gesture', 'only_gaze', 'only_gesture', 'only_objects', 'wrong_gaze_gesture']
view_names = ['ego_view_image', 'exo_view_image', 'top_view_image']

stat = {}
for setting_name in setting_names:
    for template_no in ['null', '1', '2', '3']:
        stat[f'{setting_name}_{template_no}_single'] = 150
        stat[f'{setting_name}_{template_no}_multi'] = 150

seleted_data = []
for i, row in df.iterrows():
    view_type = 'single'
    if(is_choose()):
        view_type = 'multi'
    
    template_no = row['instruction_template']
    if('null' in template_no):
        template_no = 'null'
    elif(('1_1' in template_no) or ('1_2' in template_no)):
        template_no = '1'
    else:
        template_no = template_no.split('_')[2]
    stat_key = f'{row["setting_name"]}_{template_no}_{view_type}'
    if(stat[stat_key]>0):
        seleted_data.append(1)
        data_id = row['id']
        if(view_type=='multi'):
            for view_name in view_names:
                filepath = row[view_name]
                filename = filepath.split('/')[-1]

                new_filename = f'{row["id"]}_{filename}'
                source_path = f'{base_dir_path}/{filepath}'
                des_path = f'{mturk_base_dir_path}/embodied_mturk_study/{new_filename}'
                try:
#                     print(source_path, des_path)
                    shutil.copy(source_path, des_path)
                except Exception as e:
                    print('file copy is not successfull', e)
                    continue

        if(view_type=='single'):
            view_name = view_names[random.randint(0, 2)]
            filepath = row[view_name]
            filename = filepath.split('/')[-1]

            new_filename = f'{row["id"]}_{filename}'
            source_path = f'{base_dir_path}/{filepath}'
            des_path = f'{mturk_base_dir_path}/embodied_mturk_study/{new_filename}'
            try:
#                 print(source_path, des_path)
                shutil.copy(source_path, des_path)
            except Exception as e:
                print('file copy is not successfull', e)
                continue

            for vn in view_names:
                if(vn!=view_name):
                    df.at[i, vn] = 'None'

        
        stat[stat_key] -= 1
    else:
        seleted_data.append(0)
    if(i%500==0):
        print(f'copy completed {i}')
        

copy completed 0
copy completed 500
copy completed 1000
copy completed 1500
copy completed 2000
copy completed 2500
copy completed 3000
copy completed 3500
copy completed 4000
copy completed 4500
copy completed 5000
copy completed 5500
copy completed 6000
copy completed 6500
copy completed 7000
copy completed 7500
copy completed 8000
copy completed 8500
copy completed 9000
copy completed 9500
copy completed 10000
copy completed 10500
copy completed 11000
copy completed 11500
copy completed 12000
copy completed 12500
copy completed 13000
copy completed 13500
copy completed 14000
copy completed 14500
copy completed 15000
copy completed 15500
copy completed 16000
copy completed 16500
copy completed 17000
copy completed 17500
copy completed 18000
copy completed 18500
copy completed 19000
copy completed 19500
copy completed 20000
copy completed 20500
copy completed 21000
copy completed 21500
copy completed 22000
copy completed 22500
copy completed 23000
copy completed 23500
copy completed 2

In [8]:
df['is_selected_for_mturk'] = seleted_data
df.head()

,id,ego_view_image,exo_view_image,top_view_image,ego_view_image_object_start_point_x,ego_view_image_object_start_point_y,ego_view_image_object_end_point_x,ego_view_image_object_end_point_y,exo_view_image_object_start_point_x,exo_view_image_object_start_point_y,...,view_name,setting_name,object_category,absolute_location_ego,absolute_location_exo,spatial_relation_ego,spatial_relation_exo,instruction_template,perspective,is_selected_for_mturk
0,4757_1,None,None,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,192.982941,410.124390,209.952682,236.83905,149.24411,...,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,template_1_2,NaN,1
1,4757_2,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/eg...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/ex...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,192.982941,410.124390,209.952682,236.83905,149.24411,...,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,exo_template_2_1,exo,1
2,4757_3,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/eg...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/ex...,5.10.2022.1.00.06.AM_4757/both_gaze_gesture/to...,375.997650,192.982941,410.124390,209.952682,236.83905,149.24411,...,top_view,both_gaze_gesture,Pistol,left,right,right of,left of,exo_template_3_2,exo,1
3,4757_4,5.10.2022.1.00.06.AM_4757/only_gaze/ego_view/c...,5.10.2022.1.00.06.AM_4757/only_gaze/exo_view/c...,5.10.2022.1.00.06.AM_4757/only_gaze/top_view/c...,379.255157,204.234802,413.606934,221.077087,236.83905,149.24411,...,top_view,only_gaze,Pistol,left,right,right of,left of,template_1_2,NaN,1
4,4757_5,None,None,5.10.2022.1.00.06.AM_4757/only_gaze/top_view/c...,379.255157,204.234802,413.606934,221.077087,236.83905,149.24411,...,top_view,only_gaze,Pistol,left,right,right of,left of,ego_template_2_1,ego,1


In [10]:
mturk_base_dir_path = '/project/CollabRoboGroup/datasets/embodied_mturk_study'
df.to_csv(f'{base_dir_path}/data.csv', index=False)